In [4]:
import random
import pandas as pd
import seaborn as sns

import matplotlib.pyplot as plt
import numpy as np
import pydot

from sklearn.externals.six import StringIO
from sklearn.tree import export_graphviz
import pydotplus
import score_solution
from sklearn.metrics import cohen_kappa_score, make_scorer



df = pd.read_csv('all_train_data.csv')
adoptionSpeed = df['AdoptionSpeed'] 
del df['AdoptionSpeed']

In [5]:
from sklearn.model_selection import train_test_split




train_data, test_data, adoptionSpeed, correct_ans = train_test_split(df, adoptionSpeed, test_size=0.1)




del train_data["Name"]
del train_data["RescuerID"]
del train_data["Description"]
del train_data["PetID"]


del test_data["Name"]
del test_data["RescuerID"]
del test_data["Description"]
del test_data["PetID"]

train_data = train_data.fillna(0)
test_data = test_data.fillna(0)




In [6]:
from sklearn.datasets import load_boston


In [7]:
train_data.shape

(13493, 34)

In [10]:

kappa_eval = make_scorer(cohen_kappa_score)


In [13]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from xgboost import XGBClassifier


# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 300, stop = 1000, num = 7)]
# Number of features to consider at every split
max_depths = [int(x) for x in np.linspace(3, 6, num = 1)]
learning_rate = [float(x) for x in np.linspace(0.01, 0.1, num = 0.02)]
subsample = [0.8, 0.9, 1]
# Minimum number of samples required at each leaf node
colsample_bytree = [float(x) for x in np.linspace(0.3, 0.8, num = 0.2)]
# Method of selecting samples for training each tree
gamma = [0, 1, 5]


random_grid = {
    'learning_rate': learning_rate,
    'max_depth': max_depths,
    'subsample': subsample,
    'colsample_bytree': colsample_bytree,
    'gamma': gamma,
    'n_estimators': n_estimators}





random_grid




/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: DeprecationWarning: object of type <class 'float'> cannot be safely interpreted as an integer.
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: DeprecationWarning: object of type <class 'float'> cannot be safely interpreted as an integer.
  del sys.path[0]


{'learning_rate': [],
 'max_depth': [3],
 'subsample': [0.8, 0.9, 1],
 'colsample_bytree': [],
 'gamma': [0, 1, 5],
 'n_estimators': [300, 416, 533, 650, 766, 883, 1000]}

In [14]:
xgb_model = XGBClassifier()
xgb_random = RandomizedSearchCV(
    estimator = xgb_model, param_distributions = random_grid,
     n_iter = 100, cv = 3, verbose=2, random_state=13,
     n_jobs = 4, scoring=kappa_eval
)
# Fit the random search model
xgb_random.fit(train_data, adoptionSpeed)
best_random = rf_random.best_estimator_
print(best_random)
ans = best_random.predict(test_data)
best_random
kappa_score = cohen_kappa_score(ans,correct_ans)
kappa_score


Fitting 3 folds for each of 0 candidates, totalling 0 fits


/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_search.py:271: UserWarning: The total space of parameters 0 is smaller than n_iter=100. Running 0 iterations. For exhaustive searches, use GridSearchCV.
  % (grid_size, self.n_iter, grid_size), UserWarning)
[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   0 out of   0 | elapsed:    0.0s finished


ValueError: not enough values to unpack (expected 5, got 0)

NameError: name 'ans' is not defined

In [30]:
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.datasets import make_blobs

bagging_model = BaggingClassifier(DecisionTreeClassifier(), n_estimators = 500,
                            max_samples=0.5, max_features=0.5)


bagging_model.fit(train_data, adoptionSpeed)
ans = bagging_model.predict(test_data)
kappa_score = score_solution.kappa(ans,correct_ans)
kappa_score




0.19953336025485735

In [9]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV

# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = , stop = 1000, num = 7)]
# Number of features to consider at every split
max_features = ['log2', 'sqrt']
# Maximum number of levels in tree
max_depths = [int(x) for x in np.linspace(20, 60, num = 5)]
max_depths.append(None)
# Minimum number of samples required to split a node
min_samples_splits = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leafs = [1, 2, 4]
# Method of selecting samples for training each tree
bootstraps = [True, False]


random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depths,
               'min_samples_split': min_samples_splits,
               'min_samples_leaf': min_samples_leafs,
               'bootstrap': bootstraps}






kappa_eval = make_scorer(cohen_kappa_score)


rf = RandomForestClassifier()
rf_random = RandomizedSearchCV(
    estimator = rf, param_distributions = random_grid,
     n_iter = 100, cv = 3, verbose=2, random_state=13,
     n_jobs = 4, scoring=kappa_eval
)
# Fit the random search model
rf_random.fit(train_data, adoptionSpeed)
best_random = rf_random.best_estimator_
print(best_random)
ans = best_random.predict(test_data)

kappa_score = score_solution.kappa(ans,correct_ans)
kappa_score

best_random

Fitting 3 folds for each of 100 candidates, totalling 300 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  33 tasks      | elapsed:  3.3min
[Parallel(n_jobs=4)]: Done 154 tasks      | elapsed: 18.9min
[Parallel(n_jobs=4)]: Done 300 out of 300 | elapsed: 35.7min finished


RandomForestClassifier(bootstrap=False, class_weight=None, criterion='gini',
            max_depth=None, max_features='sqrt', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=2, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=1000, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)


RandomForestClassifier(bootstrap=False, class_weight=None, criterion='gini',
            max_depth=None, max_features='sqrt', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=2, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=1000, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [10]:
kappa_score

# from sklearn.ensemble import ExtraTreesClassifier


# extra_model = ExtraTreesClassifier(bootstrap=True, class_weight=None, criterion='gini',
#             max_depth=40, max_features='log2', max_leaf_nodes=None,
#             min_impurity_decrease=0.0, min_impurity_split=None,
#             min_samples_leaf=2, min_samples_split=6,
#             min_weight_fraction_leaf=0.0, n_estimators=630, n_jobs=None,
#             oob_score=False, random_state=None, verbose=0,
#             warm_start=False)

# extra_model.fit(train_data, adoptionSpeed)

# ans = extra_model.predict(test_data)

# kappa_score = score_solution.kappa(ans,correct_ans)
# kappa_score





0.23556388352643143

In [49]:
from sklearn.ensemble import AdaBoostClassifier


agb_model = AdaBoostClassifier(learning_rate=0.1, n_estimators=1000,
                           algorithm='SAMME.R', base_estimator=DecisionTreeClassifier(max_depth=2))


agb_model.fit(train_data, adoptionSpeed)

ans = agb_model.predict(test_data)

kappa_score = score_solution.kappa(ans,correct_ans)
kappa_score



0.19660782051518066

In [50]:
from sklearn.ensemble import GradientBoostingClassifier

gbc_model = GradientBoostingClassifier(n_estimators=1000, learning_rate=0.1,
     max_depth=2)

gbc_model.fit(train_data, adoptionSpeed)

ans = gbc_model.predict(test_data)

kappa_score = score_solution.kappa(ans,correct_ans)
kappa_score

0.2061097838449416

In [52]:
from sklearn.ensemble import VotingClassifier


gbc_model = GradientBoostingClassifier(n_estimators=1000, learning_rate=0.1,
     max_depth=2)
agb_model = AdaBoostClassifier(learning_rate=0.1, n_estimators=1000,
                           algorithm='SAMME.R', base_estimator=DecisionTreeClassifier(max_depth=2))
extra_model = ExtraTreesClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=40, max_features='log2', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=2, min_samples_split=6,
            min_weight_fraction_leaf=0.0, n_estimators=630, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

rf_model = RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=40, max_features='log2', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=2, min_samples_split=6,
            min_weight_fraction_leaf=0.0, n_estimators=630, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)
bagging_model = BaggingClassifier(DecisionTreeClassifier(), n_estimators = 500,
                            max_samples=0.5, max_features=0.5)

parameters = {
    'learning_rate':0.01,'max_depth':4,
    'subsample':0.8,'objective':'binary:logistic',
    'num_class':5, 'n_jobs':4,  'n_estimators':1000}

xgb_model = xgb.XGBClassifier(**parameters)

classifiers = [
    ('gbc', gbc_model), ('agb', agb_model),
    ('extra', extra_model), ('rf', rf_model),
    ('bagging', bagging_model), ("xgb", xgb_model)
]

voting_model = VotingClassifier(estimators=classifiers, voting='hard')

voting_model.fit(train_data, adoptionSpeed)

ans = voting_model.predict(test_data)

kappa_score = score_solution.kappa(ans,correct_ans)
kappa_score


0.22060947803154352